# Draw histograms

In [2]:
import numpy as np
import json
import os
import ROOT
import CMS_lumi as CMS_lumi 
import tdrstyle as tdrstyle
from collections import OrderedDict

%jsroot on
ROOT.gROOT.ForceStyle()
tdrstyle.setTDRStyle()
ROOT.gStyle.SetOptStat(0)

outputFolder = 'Plots/'
if not os.path.exists(outputFolder):
    os.mkdir(outputFolder)
    print("Directory " , outputFolder ,  " Created ")
else:    
    print("Directory " , outputFolder ,  " already exists")

Welcome to JupyROOT 6.18/00
Directory  Plots/  already exists


## Load rootfiles

In [3]:
versionRootfiles = 'v02'
lumi=41529

bkgFiles = {}
bkgFiles["ST_tW_antitop"] = [ ROOT.TFile('Rootfiles/'+versionRootfiles+'/histograms_ST_tW_antitop_5f_inclusiveDecays_TuneCP5_PSweights_13TeV-powheg-pythia8.root'), 1 ]
bkgFiles["ST_tW_top"] = [ ROOT.TFile('Rootfiles/'+versionRootfiles+'/histograms_ST_tW_top_5f_inclusiveDecays_TuneCP5_PSweights_13TeV-powheg-pythia8.root'), 1 ]
bkgFiles["TTTo2L2Nu"] = [ ROOT.TFile('Rootfiles/'+versionRootfiles+'/histograms_TTTo2L2Nu_TuneCP5_13TeV-powheg-pythia8.root'), lumi*88.342/283000430.596, 28 ]
bkgFiles["TTToHadronic"] = [ ROOT.TFile('Rootfiles/'+versionRootfiles+'/histograms_TTToHadronic_TuneCP5_13TeV-powheg-pythia8.root'), 1, 28 ]
bkgFiles["TTToSemiLeptonic"] = [ ROOT.TFile('Rootfiles/'+versionRootfiles+'/histograms_TTToSemiLeptonic_TuneCP5_13TeV-powheg-pythia8.root'), lumi*365.46/720253370.04, 27 ]
bkgFiles["TTWJetsToQQ"] = [ ROOT.TFile('Rootfiles/'+versionRootfiles+'/histograms_TTWJetsToQQ_TuneCP5_13TeV-amcatnloFXFX-madspin-pythia8.root'), 1 ]
bkgFiles["TTZToQQ"] = [ ROOT.TFile('Rootfiles/'+versionRootfiles+'/histograms_TTZToQQ_TuneCP5_13TeV-amcatnlo-pythia8.root'), 1 ]
bkgFiles["WW"] = [ ROOT.TFile('Rootfiles/'+versionRootfiles+'/histograms_WW_TuneCP5_13TeV-pythia8.root'), 1 ]
#bkgFiles[""] = [ ROOT.TFile('Rootfiles/'+versionRootfiles+'/'), 1 ]
#bkgFiles[""] = [ ROOT.TFile('Rootfiles/'+versionRootfiles+'/'), 1 ]

signalFiles = {}
signalFiles["ttHToNonbb"] = [ ROOT.TFile('Rootfiles/'+versionRootfiles+'/histograms_ttHToNonbb_M125_TuneCP5_13TeV-powheg-pythia8.root'), 1 ]
signalFiles["ttHTobb"] = [ ROOT.TFile('Rootfiles/'+versionRootfiles+'/histograms_ttHTobb_M125_TuneCP5_13TeV-powheg-pythia8.root'), lumi*0.2934045/4216319.32, 9 ]
signalFiles["ttHTobb_ttToSemiLep"] = [ ROOT.TFile('Rootfiles/'+versionRootfiles+'/histograms_ttHTobb_ttToSemiLep_M125_TuneCP5_13TeV-powheg-pythia8.root'), 1 ]
#signalFiles[""] = [ ROOT.TFile('Rootfiles/'+versionRootfiles+'/'), 1 ]

### Helpful functions

In [4]:
def addingText( listSel, xMin=0.65, yMax=0.65, align='right' ):

    for i in range( len( listSel ) ):
        textBox=ROOT.TLatex()
        textBox.SetNDC()
        textBox.SetTextSize(0.04) 
        #if 'right' in align: textBox.SetTextAlign(31)
        textBox.SetTextFont(62) ### 62 is bold, 42 is normal
        textBox.DrawLatex(xMin, yMax, listSel[i])
        yMax = yMax -0.05

## Single plots

In [5]:
canvas = {}
def plotSimple( inFile, sample, name, rebinX='', xmin='', xmax='', labX=0.92, labY=0.50, axisX='', axisY='', log=False, ext='png', version=versionRootfiles, Norm=False ):
    """"Take root file, make simple plot"""
    
    outputFileName = name+'_'+sample+'_simplePlot'+version+'.'+ext
    print('Processing.......', outputFileName)
    
    histo = inFile.Get( 'tthbb13/'+name )
    
    binWidth = histo.GetBinWidth(1)

    histo.SetLineColor(ROOT.kBlue)
    histo.SetLineWidth(2)
    histo.SetFillStyle(1001)
    histo.SetTitle(sample+' '+name)
    if rebinX: histo.Rebin( rebinX )

    #tdrstyle.SetPadRightMargin(0.05)
    canvas[name] = ROOT.TCanvas('c'+name+sample, 'c'+name+sample, 10, 10, 750, 500 )
    
    if log: 
        canvas[name].SetLogy()
        outName = outputFileName.replace('_simplePlot','_Log_simplePlot')
    else: outName = outputFileName 
    canvas[name].Draw()

    legend=ROOT.TLegend(0.60,0.75,0.90,0.90)
    legend.SetFillStyle(0)
    legend.SetTextSize(0.04)
    legend.AddEntry( histo, sample, 'l' )
    
    if xmax: histo.GetXaxis().SetRangeUser( xmin, xmax )
    histo.GetYaxis().SetTitleOffset(0.90)
    histo.Draw('hist')
    if not axisY: histo.GetYaxis().SetTitle( 'Events / '+str(binWidth) )
    if axisX: histo.GetXaxis().SetTitle( axisX )

    CMS_lumi.extraText = "Simulation Preliminary"
    CMS_lumi.lumi_13TeV = ''
    CMS_lumi.CMS_lumi(canvas[name], 4, 0)

    legend.Draw()
    #addingText( sample, labX, labY, 'right')
    canvas[name].Modified()
    canvas[name].Update()
    canvas[name].SaveAs( 'Plots/'+outName )
    del canvas[name]

In [7]:
histos = [
    ['ngoodEvents', 'Categories', 0, 8, 1, True ],
    ['TopCandMass', 'Top candidate mass [GeV]', 30, 300, 1, False ],
    ['HiggsCandMass', 'Higgs candidate mass [GeV]', 30, 250, 1, False ],
    ['WCandMass', 'W candidate mass [GeV]', 30, 200, 1, False ],
    ['HiggsCandMass_WHiggs', 'Higgs candidate mass [GeV]', 30, 250, 1, False ],
    ['WCandMass_WHiggs', 'W candidate mass [GeV]', 30, 200, 1, False ],
    ['lepTopCandMass_boostedW', 'Top candidate mass [GeV]', 30, 300, 1, False ],
    ['hadTopCandMass_boostedW', 'Top candidate mass [GeV]', 30, 300, 1, False ],
    ['boostedHiggsCandMass_boostedW', 'Higgs candidate mass [GeV]', 30, 250, 1, False ],
    ['resolvedHiggsCandMass_boostedW', 'Higgs candidate mass [GeV]', 30, 250, 1, False ],
]

isample = {}
isample['ttHTobb'] = signalFiles['ttHTobb']
isample["TTToSemiLeptonic"] = bkgFiles["TTToSemiLeptonic"]

for h in histos:
    for s in isample:
        print(h[0])
        plotSimple( isample[s][0], s, h[0], axisX=h[1], xmin=h[2], xmax=h[3], rebinX=h[4], log=h[5])


ngoodEvents
Processing....... ngoodEvents_ttHTobb_simplePlotv02.png
ngoodEvents
Processing....... ngoodEvents_TTToSemiLeptonic_simplePlotv02.png
TopCandMass
Processing....... TopCandMass_ttHTobb_simplePlotv02.png
TopCandMass
Processing....... TopCandMass_TTToSemiLeptonic_simplePlotv02.png
HiggsCandMass
Processing....... HiggsCandMass_ttHTobb_simplePlotv02.png
HiggsCandMass
Processing....... HiggsCandMass_TTToSemiLeptonic_simplePlotv02.png
WCandMass
Processing....... WCandMass_ttHTobb_simplePlotv02.png
WCandMass
Processing....... WCandMass_TTToSemiLeptonic_simplePlotv02.png
HiggsCandMass_WHiggs
Processing....... HiggsCandMass_WHiggs_ttHTobb_simplePlotv02.png
HiggsCandMass_WHiggs
Processing....... HiggsCandMass_WHiggs_TTToSemiLeptonic_simplePlotv02.png
WCandMass_WHiggs
Processing....... WCandMass_WHiggs_ttHTobb_simplePlotv02.png
WCandMass_WHiggs
Processing....... WCandMass_WHiggs_TTToSemiLeptonic_simplePlotv02.png
lepTopCandMass_boostedW
Processing....... lepTopCandMass_boostedW_ttHTobb_

Warning in <TCanvas::Constructor>: Deleting canvas with same name: cngoodEventsttHTobb
Info in <TCanvas::Print>: png file Plots/ngoodEvents_ttHTobb_Log_simplePlotv02.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: cngoodEventsTTToSemiLeptonic
Info in <TCanvas::Print>: png file Plots/ngoodEvents_TTToSemiLeptonic_Log_simplePlotv02.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: cTopCandMassttHTobb
Info in <TCanvas::Print>: png file Plots/TopCandMass_ttHTobb_simplePlotv02.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: cTopCandMassTTToSemiLeptonic
Info in <TCanvas::Print>: png file Plots/TopCandMass_TTToSemiLeptonic_simplePlotv02.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: cHiggsCandMassttHTobb
Info in <TCanvas::Print>: png file Plots/HiggsCandMass_ttHTobb_simplePlotv02.png has been created
Warning in <TCanvas::Constructor>: De

### Signal and background plots

In [6]:
canvas = {}
#def plotSimple( inFile, sample, name, rebinX='', xmin='', xmax='', labX=0.92, labY=0.50, axisX='', axisY='', log=False, ext='png', version=versionRootfiles, Norm=False ):
def plotSignalBkg( signalFiles, bkgFiles, name, xmin, xmax, rebinX, labX=0.92, labY=0.50, log=False, 
                      version=versionRootfiles, addRatioFit=False, Norm=False, ext='png' ):
    """function to plot s and b histos"""

    outputFileName = name+'_PlusBkg_AnalysisPlots_'+version+'.'+ext 
    if log: outputFileName = outputFileName.replace('Plots','Plots_Log')
    print('Processing.......', outputFileName)

    legend=ROOT.TLegend(0.60,0.75,0.90,0.90)
    legend.SetFillStyle(0)
    legend.SetTextSize(0.04)

#	if 'DATA' in args.process:
#		dataHistos = {}
#		dataHistos[ 'DATA' ] = dataFile.Get( nameInRoot+'_JetHT_Run2016'+tmpRegion if args.miniTree else args.boosted+'AnalysisPlots'+('' if 'pruned' in args.grooming else args.grooming)+'/'+nameInRoot  )
#		if 'massAve' in nameInRoot: 
#			dataHistos[ 'DATA' ] = dataHistos[ 'DATA' ].Rebin( len( boostedMassAveBins )-1, dataHistos[ 'DATA' ].GetName(), boostedMassAveBins )
#			dataHistos[ 'DATA' ].Scale ( 1, 'width' )
#		elif rebinX > 1: dataHistos[ 'DATA' ] = dataHistos[ 'DATA' ].Rebin( rebinX )
#        legend.AddEntry( dataHistos[ 'DATA' ], 'Data', 'lep' )
#        if Norm: dataHistos[ 'DATA' ].Scale( 1 /dataHistos['DATA'].Integral() )

    bkgHistos = OrderedDict()
    binWidth = 0
    maxList = []
    bkgInMassWindow = 0
    bkgInMassWindowErr = 0
    if len(bkgFiles) > 0:
        for bkgSamples in bkgFiles:
            bkgHistos[ bkgSamples ] = bkgFiles[ bkgSamples ][0].Get( 'tthbb13/'+name )
            bkgHistos[ bkgSamples ].SetTitle(bkgSamples)
            if bkgFiles[ bkgSamples ][1] != 1: bkgHistos[ bkgSamples ].Scale( bkgFiles[ bkgSamples ][1] )
            print(bkgHistos[ bkgSamples ].Integral())
            if rebinX > 1: bkgHistos[ bkgSamples ] = bkgHistos[ bkgSamples ].Rebin( rebinX )
            legend.AddEntry( bkgHistos[ bkgSamples ], bkgSamples, 'l' if Norm else 'f' )

            if Norm:
                bkgHistos[ bkgSamples ].SetLineColor( bkgFiles[ bkgSamples ][2] )
                bkgHistos[ bkgSamples ].SetLineWidth( 2 )
                bkgHistos[ bkgSamples ].Scale( 1 / bkgHistos[ bkgSamples ].Integral() )
                maxList.append( bkgHistos[ bkgSamples ].GetMaximum() )
            else:
                bkgHistos[ bkgSamples ].SetFillStyle( 1001 )
                bkgHistos[ bkgSamples ].SetFillColor( int(bkgFiles[ bkgSamples ][2]) )

    signalHistos = OrderedDict()
    if len(signalFiles) > 0:
        dummySig=0
        for sigSamples in signalFiles:
            signalHistos[ sigSamples ] = signalFiles[ sigSamples ][0].Get( 'tthbb13/'+name )
            if signalFiles[ sigSamples ][1] != 1: signalHistos[ sigSamples ].Scale( signalFiles[ sigSamples ][1] ) 
            legend.AddEntry( signalHistos[ sigSamples ], sigSamples, 'l' if Norm else 'f' )
#			if 'massAve' in nameInRoot: 
#				signalHistos[ sigSamples ].Scale( twoProngSF * antiTau32SF ) 
#				signalHistos[ sigSamples ] = signalHistos[ sigSamples ].Rebin( len( boostedMassAveBins )-1, signalHistos[ sigSamples ].GetName(), boostedMassAveBins )
#				signalHistos[ sigSamples ].Scale ( 1, 'width' )
#				totalIntegralSig = signalHistos[ sigSamples ].Integral()
#				nEntriesTotalSig = signalHistos[ sigSamples ].GetEntries()
#				totalSF = totalIntegralSig/nEntriesTotalSig
#				windowIntegralSigErr = Double(0)
#				windowIntegralSig = signalHistos[ sigSamples ].IntegralAndError((args.mass-10)/rebinX, (args.mass+10)/rebinX, windowIntegralSigErr ) 
#				print sigSamples, round(totalIntegralSig,2), nEntriesTotalSig, totalSF
#				print sigSamples, 'in mass window', round(windowIntegralSig,2), ', nEntries', windowIntegralSig/totalSF, windowIntegralSigErr
            if rebinX > 1: signalHistos[ sigSamples ] = signalHistos[ sigSamples ].Rebin( rebinX )
            if Norm:
                signalHistos[ sigSamples ].SetLineColor( signalFiles[ sigSamples ][2] )
                signalHistos[ sigSamples ].SetLineWidth( 3 )
                signalHistos[ sigSamples ].SetLineStyle( 10-dummySig )
                signalHistos[ sigSamples ].Scale( 1 / signalHistos[ sigSamples ].Integral() )
                maxList.append( signalHistos[ sigSamples ].GetMaximum() )
            else:
#				if 'DATA' in args.process:
#					signalHistos[ sigSamples ].SetLineColor( signalFiles[ sigSamples ][3] )
#					signalHistos[ sigSamples ].SetFillColor(0)
#					signalHistos[ sigSamples ].SetLineWidth(3)
#					signalHistos[ sigSamples ].SetLineStyle(2+dummySig)
#				else:
                signalHistos[ sigSamples ].SetFillStyle( 1001 )
                signalHistos[ sigSamples ].SetFillColor( signalFiles[ sigSamples ][2] )
                signalHistos[ sigSamples ].SetLineColor( signalFiles[ sigSamples ][2] )
            binWidth = int(signalHistos[ sigSamples ].GetBinWidth( 1 ))
            dummySig+=8


    hBkg = bkgHistos[next(iter(bkgHistos))].Clone()
    hBkg.Reset()

    if not Norm:

        stackHisto = ROOT.THStack('stackHisto'+name, 'stack'+name)
        for samples in bkgHistos: 
            stackHisto.Add( bkgHistos[ samples ].Clone() )
            hBkg.Add( bkgHistos[ samples ].Clone() )

        canvas[outputFileName] = ROOT.TCanvas('c1'+name, 'c1'+name,  10, 10, 750, 500 )
        #tdrStyle.SetPadRightMargin(0.05)
        #tdrStyle.SetPadLeftMargin(0.15)
        #pad1 = ROOT.TPad("pad1", "Fit",0,0.207,1.00,1.00,-1)
        #pad2 = ROOT.TPad("pad2", "Pull",0,0.00,1.00,0.30,-1);
        #pad1.Draw()
        #pad2.Draw()

        #pad1.cd()
        #if log and not args.final: pad1.SetLogy()
        if log: can.SetLogy()
        stackHisto.Draw('hist')

        if xmax: stackHisto.GetXaxis().SetRangeUser( xmin, xmax )
        stackHisto.SetMaximum( hBkg.GetMaximum()*1.2 ) 
        stackHisto.GetYaxis().SetTitleOffset( 0.8 )

        #stackHisto.SetMinimum( 0.1 ) 
        #hBkg.SetFillStyle(0)
        hBkg.SetLineColor(ROOT.kBlack)
        hBkg.SetLineStyle(1)
        hBkg.SetLineWidth(1)
        #hBkg.SetFillStyle(3004)
        #hBkg.SetFillColor( kRed )
        #hBkg.Draw("same")

        stackHisto.GetYaxis().SetTitle( 'Events / '+str(binWidth)+' GeV' )

#		if 'DATA' in args.process: 
#			dataHistos[ 'DATA' ].SetMarkerStyle(8)
#			dataHistos[ 'DATA' ].Draw('same')
#			CMS_lumi.extraText = ""#"Preliminary"
#			legend.SetNColumns(2)
#			if not 'Tau32' in args.cut:
#				for sample in signalHistos: 
#					if 'massAve' in nameInRoot: 
#						#lowEdgeWindow = int(int(sample) - ( int( massWidthList[int(sample)])*3 ))
#						#highEdgeWindow = int(int(sample) + ( int( massWidthList[int(sample)])*3 ))
#						tmpResolution = 2*(-1.78 + ( 0.1097 * int(sample)) + ( -0.0002897 * int(sample)*int(sample) ) + ( 3.18e-07 * int(sample)*int(sample)*int(sample)))
#						lowEdgeWindow = int(int(sample) - tmpResolution )
#						highEdgeWindow = int(int(sample) + tmpResolution )
#						signalHistos[ sample ].GetXaxis().SetRangeUser( lowEdgeWindow, highEdgeWindow )
#					signalHistos[ sample ].Draw("hist same")
#		else:  
        
        tmpHisto = {}
        for sample in signalHistos: 
            tmpHisto[ sample ] = signalHistos[ sample ].Clone()
            tmpHisto[ sample ].SetFillColor(0)
            tmpHisto[ sample ].SetLineStyle(2)
            tmpHisto[ sample ].SetLineWidth(3)
            tmpHisto[ sample ].Draw("hist same")

        CMS_lumi.extraText = "Simulation Preliminary"
        CMS_lumi.lumi_13TeV = str( round( (lumi/1000.), 1 ) )+" fb^{-1}"
        #CMS_lumi.relPosX = 0.14
        CMS_lumi.CMS_lumi( canvas[outputFileName], 4, 0)
        legend.Draw()

#         if not args.final:
# 			pad2.cd()
# 			pad2.SetGrid()
# 			pad2.SetTopMargin(0)
# 			pad2.SetBottomMargin(0.3)
			
# 			if 'DATA' in args.process:
# 				tmpPad2= pad2.DrawFrame(xmin,0.5,xmax,1.5)
# 				labelAxis( name.replace( args.cut, ''), tmpPad2, ( 'softDrop' if 'Puppi' in args.grooming else args.grooming ) )
# 				tmpPad2.GetYaxis().SetTitle( "Data/Bkg" )
# 				tmpPad2.GetYaxis().SetTitleOffset( 0.5 )
# 				tmpPad2.GetYaxis().CenterTitle()
# 				tmpPad2.SetLabelSize(0.12, 'x')
# 				tmpPad2.SetTitleSize(0.12, 'x')
# 				tmpPad2.SetLabelSize(0.12, 'y')
# 				tmpPad2.SetTitleSize(0.12, 'y')
# 				tmpPad2.SetNdivisions(505, 'x')
# 				tmpPad2.SetNdivisions(505, 'y')
# 				pad2.Modified()
# 				hRatio = TGraphAsymmErrors()
# 				hRatio.Divide( dataHistos[ 'DATA' ], hBkg, 'pois' )
# 				hRatio.SetMarkerStyle(8)
# 				hRatio.Draw('P')
				
# 			else:
# 				hRatio = signalHistos[ args.mass ].Clone()
# 				hRatio.Reset()
# 				allBkgWindow = 0
# 				allSigWindow = 0
# 				for ibin in range((args.mass-10)/rebinX, (args.mass+10)/rebinX+1 ): 
# 					binContSignal = signalHistos[ args.mass ].GetBinContent(ibin)
# 					allSigWindow += binContSignal
# 					binContBkg = hBkg.GetBinContent(ibin)
# 					allBkgWindow += binContBkg
# 					try: value = binContSignal / TMath.Sqrt( binContBkg )
# 					#try: value = binContSignal / TMath.Sqrt( binContSignal + binContBkg )
# 					#try: value = binContSignal / ( binContSignal + binContBkg )
# 					except ZeroDivisionError: continue
# 					hRatio.SetBinContent( ibin, value )
# 				ratioLabel = "S / #sqrt{B}"
# 				print 's/sqrt(B) ', allSigWindow/TMath.Sqrt(allBkgWindow), allSigWindow, allBkgWindow, allSigWindow/allBkgWindow
# 				print '2 ( sqrt(B+S) - sqrt(B) )', 2*( TMath.Sqrt( allBkgWindow+allSigWindow ) - TMath.Sqrt( allBkgWindow ) ) 
			
# 				labelAxis( name, hRatio, ( 'softDrop' if 'Puppi' in args.grooming else args.grooming) )
# 				hRatio.GetYaxis().SetTitleOffset(1.2)
# 				hRatio.GetXaxis().SetLabelSize(0.12)
# 				hRatio.GetXaxis().SetTitleSize(0.12)
# 				hRatio.GetYaxis().SetTitle( ratioLabel )
# 				hRatio.GetYaxis().SetLabelSize(0.12)
# 				hRatio.GetYaxis().SetTitleSize(0.12)
# 				hRatio.GetYaxis().SetTitleOffset(0.45)
# 				hRatio.GetYaxis().CenterTitle()
# 				#hRatio.SetMaximum(0.7)
# 				if xmax: hRatio.GetXaxis().SetRangeUser( xmin, xmax )
# 				hRatio.Draw( ("PES" if 'DATA' in args.process else "hist" ) )

# 			if addRatioFit:
# 				tmpFit = TF1( 'tmpFit', 'pol0', 120, 240 )
# 				hRatio.Fit( 'tmpFit', '', '', 120, 240 )
# 				tmpFit.SetLineColor( kGreen )
# 				tmpFit.SetLineWidth( 2 )
# 				tmpFit.Draw("sames")
# 				chi2Test = TLatex( 0.7, 0.8, '#splitline{#chi^{2}/ndF = '+ str( round( tmpFit.GetChisquare(), 2 ) )+'/'+str( int( tmpFit.GetNDF() ) )+'}{p0 = '+ str( round( tmpFit.GetParameter( 0 ), 2 ) ) +' #pm '+str(  round( tmpFit.GetParError( 0 ), 2 ) )+'}' )
# 				chi2Test.SetNDC()
# 				chi2Test.SetTextFont(42) ### 62 is bold, 42 is normal
# 				chi2Test.SetTextSize(0.10)
# 				chi2Test.Draw('same')

# 			if 'DATA' in args.process: 
# 				hRatio.GetYaxis().SetNdivisions(505)
# 				line.Draw('same')

        canvas[outputFileName].SaveAs( 'Plots/'+outputFileName )
        #del canvas[outputFileName]

In [7]:
histos = [
    ['HiggsCandMass_WHiggs', 'Higgs candidate mass [GeV]', 30, 250, 1, False ],
    ['WCandMass_WHiggs', 'W candidate mass [GeV]', 30, 200, 1, False ],
]

tmpSig = {'ttHTobb': signalFiles['ttHTobb']}
tmpBkg = {k: bkgFiles[k] for k in ('TTTo2L2Nu', 'TTToSemiLeptonic' )}

for h in histos:
    print(h[0])
    plotSignalBkg( tmpSig, tmpBkg, h[0], h[2], h[3], h[4], log=h[5] )
ROOT.gROOT.GetListOfCanvases().Draw()

HiggsCandMass_WHiggs
Processing....... HiggsCandMass_WHiggs_PlusBkg_AnalysisPlots_v02.png
79.77909323200583
452.8164862021804
WCandMass_WHiggs
Processing....... WCandMass_WHiggs_PlusBkg_AnalysisPlots_v02.png
78.88459284044802
431.2176784276962


Info in <TCanvas::Print>: png file Plots/HiggsCandMass_WHiggs_PlusBkg_AnalysisPlots_v02.png has been created
Info in <TCanvas::Print>: png file Plots/WCandMass_WHiggs_PlusBkg_AnalysisPlots_v02.png has been created
